In [1]:
from icrl import *
from model import *
import pickle

d:\Anaconda3\envs\textgen\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin d:\Anaconda3\envs\textgen\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
# load trajectory pkl
traj = pickle.load(open('data/linucb_trajectories.pkl', 'rb'))

In [3]:
import wandb

wandb.login(key = 'da9f2bbc59300d7434138182a83a9b35c0cea793') # login to wandb
wandb.init(project="icrl_pretrain", reinit=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: xjxyys. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Lenovo/.netrc


In [8]:
# split trajectory into train and test
import tqdm
from sklearn.model_selection import train_test_split

train_traj, test_traj = train_test_split(traj, test_size=0.2)

batch_size = 8

# create dataset
train_dataset = TrajectoryDataset(train_traj)
test_dataset = TrajectoryDataset(test_traj)

# create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# set up model
model = TransformerModel(embed_dim=70, num_heads=5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5*batch_size)

# Trainer
def trainer(model,train_dataloader, test_dataloader, optimizer, num_epochs=10):
    # DataLoader can handle batching and shuffling
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}")
        epoch_loss = 0.0
        for batch in tqdm.tqdm(train_dataloader):
            # Unpack the data
            tokens, action_labels = batch
            tokens  = tokens.to(device)
            action_labels = action_labels.to(device)
            
            # Reset the gradients in the optimizer
            optimizer.zero_grad()
            # type
            # Forward pass
            logits = model(tokens)  # Your forward method should handle the causal mask internally
            
            # Compute loss
            loss = loss_fn(logits, action_labels)
    
            # Backward pass
            loss.backward()
            
            # Update parameters
            optimizer.step()
            
            epoch_loss += loss.item()
        epoch_loss /= len(train_dataloader)
        # Compute the test loss
        test_loss = 0.0
        model.eval()
        with torch.no_grad():
            for batch in tqdm.tqdm(test_dataloader):
                tokens, action_labels = batch
                tokens  = tokens.to(device)
                action_labels = action_labels.to(device)
                logits = model(tokens)
                loss = loss_fn(logits, action_labels)
                test_loss += loss.item()
        model.train()
        test_loss /= len(test_dataloader)
        # wandb logging
        wandb.log({"train_loss": epoch_loss, "test_loss": test_loss})
        print(f"Epoch {epoch} | Train Loss {epoch_loss} | Test Loss {test_loss}")


Using device cuda


In [9]:
trainer(model, train_dataloader, test_dataloader, optimizer, num_epochs=10)

Epoch 0


 75%|███████▍  | 748/1000 [01:41<00:36,  6.81it/s]

In [6]:
# save model
torch.save(model.state_dict(), 'model/transformer_model.pth')

In [ ]:
# evaluate model
